In [8]:
import numpy as np
import dask.array as da
from dask.distributed import Client
from functions import kernel_est_funcs as kef

In [9]:
spikes = np.load('data/spikes-10e4-ms.npy')

In [14]:
# Set up a Dask client for parallelism
client = Client()
'''
# Define Process 1
def process_1(block):
    print(f"Process 1 running on chunk with shape {block.shape}")
    return block * 2

# Define Process 2
def process_2(block):
    print(f"Process 2 running on chunk with shape {block.shape}")
    return block + 5

# Create a large Dask array with chunks
large_array = da.random.random((1000, 100000), chunks=(20, 10000))

# Apply Process 1 using map_blocks
processed_1 = large_array.map_blocks(process_1)

# Apply Process 2 using map_blocks on the result of Process 1
processed_2 = processed_1.map_blocks(process_2)

# Trigger computation and track progress
result = processed_2.compute()
'''
dask_array = da.from_array(spikes, chunks=(1250, 10000))
pr1 = dask_array.map_blocks(kef.sim_calcium, neuron_id=-1)
pr2 = pr1.map_blocks(kef.smoothed_signals, win_len=5)
result = pr2.compute()

# Check the result
print("Processing completed!")


/home/mjoudy/Documents/codes/0-PhD Project/Calcium---AR/phd-env/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42097 instead
  warnings.warn(


ValueError: `dtype` inference failed in `map_blocks`.

Please specify the dtype explicitly using the `dtype` kwarg.

Original error is below:
------------------------
IndexError('index 500 is out of bounds for axis 0 with size 1')

Traceback:
---------
  File "/home/mjoudy/Documents/codes/0-PhD Project/Calcium---AR/phd-env/lib/python3.12/site-packages/dask/array/core.py", line 459, in apply_infer_dtype
    o = func(*args, **kwargs)
        ^^^^^^^^^^^^^^^^^^^^^
  File "/home/mjoudy/Documents/codes/0-PhD Project/Calcium---AR/notebooks - analysis/functions/kernel_est_funcs.py", line 50, in sim_calcium
    spikes = spikes[neuron_id, wup_time:]
             ~~~~~~^^^^^^^^^^^^^^^^^^^^^^
